<a href="https://colab.research.google.com/github/pranavgupta0001/my.chat/blob/main/test/cfc_rag_0001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#details
The simple rag on CropManage data, just through the data in and it will take produce the answers.

#imports


In [2]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00


In [3]:
pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [4]:
pip install -qU langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.1 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
import getpass
import os
from langchain_openai import ChatOpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import json

#Credentials

In [6]:
# Set your environment variables for LangChain and OpenAI
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Enables LangSmith (LangChain) tracing if needed
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

#model and embeddings

In [7]:
# Initialize your model and embeddings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#Data Loader


In [8]:
def json_to_langchain_full_docs(file_path: str) -> list[Document]:
    """
    Reads a CropManage JSON file, extracts the resource_description
    and the main data content, then returns a list of LangChain Documents.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    data_str = "All Data:\n" + json.dumps(data)
    doc = Document(
        page_content=data_str,
        metadata={
            "file_name": file_path
        }
    )

    return [doc]

All Data:
{"CommodityType": {"endpoint": "https://api.dev.cropmanage.ucanr.edu/v2/commodity-types.json", "description": "Returns a complete list of commodity types in CropManage.", "resource_description": {"inputs_description": {}, "outputs_description": {"Id": {"description": "commodity type ID", "type": "integer"}, "Name": {"description": "commodity type name", "type": "string"}, "IconPath": {"description": "commodity type icon path, used in planting settings", "type": "string"}, "HarvestOnce" ...


##Import data

In [ ]:
# Example usage for the CommodityType.json file:
file_path = "/content/CommodityType.json"  # Replace with your actual path in Google Colab or local environment
docs = json_to_langchain_full_docs(file_path)

# Inspect the text if curious:
print(docs[0].page_content[:500], "...")

##Text splitting
for better retrieval

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

split_docs = []
for doc in docs:
    for chunk in text_splitter.split_text(doc.page_content):
        split_docs.append(
            Document(page_content=chunk, metadata=doc.metadata)
        )

print(f"Number of splitted docs: {len(split_docs)}")

##Pinecone

In [9]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
# Name your Pinecone Index
index_name = "cropmanage-index"

# Initialize Pinecone client using userdata for API key (This part is correct)
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

# Ensure the index exists (create it if needed)
if index_name not in pc.list_indexes().names():
    pc.create_index(index_name, dimension=3072, metric="cosine", spec={'serverless': {'cloud': 'aws', 'region': 'us-east-1'}})

# Connect to the existing index
index = pc.Index(index_name)

# **CHANGED:** Instead of using `Pinecone.from_documents`, we instantiate `PineconeVectorStore`
# directly, passing in the existing `pc` client. This avoids the problem with LangChain's internal
# Pinecone initialization that doesn't use your API key from `userdata`.

vectorstore = PineconeVectorStore(
    embedding=embeddings,
    index=index,             # <--- pass in your pc.Index object
)

## Upsert
to Pinecone

In [10]:
_ = vectorstore.add_documents(documents=docs)

#Retriever and Generator


In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "mmr" / "similarity_score_threshold"
    search_kwargs={"k": 3}     # retrieve top 3 relevant chunks
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # or "map_reduce", "refine" for longer contexts
    retriever=retriever,
    return_source_documents=True  # useful for debugging
)

#Queries

In [12]:
# Example query 1: Harvest frequency of Walnut and Strawberry?
query = "What is the harvest frequency of Walnut and Strawberry?"
result = qa_chain({"query": query})
print("Query:", query)
print("Answer:", result["result"])
print("\nSources:")
for src in result["source_documents"]:
    print(src.metadata, "\n")

<ipython-input-12-0d7c6d9e728d>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Query: What is the harvest frequency of Walnut and Strawberry?
Answer: Walnut has a harvest frequency of "not harvested once" (it is harvested multiple times), and Strawberry has a harvest frequency of "not harvested once" (it is also harvested multiple times).

Sources:
{'file_name': '/content/CommodityType.json', 'top_level_key': 'CommodityType'} 

{'file_name': '/content/CommodityType.json', 'top_level_key': 'CommodityType'} 

{'file_name': '/content/CommodityType.json', 'top_level_key': 'CommodityType'} 



In [13]:
queries = [
    "Which crops can I manage?",
    "What are the commodity types available for my ranch in California?",
    "Is alfalfa a supported crop?",
    "Is onion a supported crop?",
    "Is watermelon harvested once or multiple times?",
    "What is the harvest frequency of watermelon?",
    "is Tomato supported and whats its harvest frequency"
]

for q in queries:
    res = qa_chain({"query": q})
    print("\nQuery:", q)
    print("Answer:", res["result"])
    print("Sources:", [d.metadata["file_name"] for d in res["source_documents"]])



Query: Which crops can I manage?
Answer: I don't know.
Sources: ['/content/CommodityType.json', '/content/CommodityType.json', '/content/CommodityType.json']

Query: What are the commodity types available for my ranch in California?
Answer: I don't know.
Sources: ['/content/CommodityType.json', '/content/CommodityType.json', '/content/CommodityType.json']

Query: Is alfalfa a supported crop?
Answer: Yes, alfalfa is a supported crop, but it is currently marked as inactive.
Sources: ['/content/CommodityType.json', '/content/CommodityType.json', '/content/CommodityType.json']

Query: Is onion a supported crop?
Answer: Yes, onion is a supported crop. It can be harvested once.
Sources: ['/content/CommodityType.json', '/content/CommodityType.json', '/content/CommodityType.json']

Query: Is watermelon harvested once or multiple times?
Answer: Watermelon is harvested once.
Sources: ['/content/CommodityType.json', '/content/CommodityType.json', '/content/CommodityType.json']

Query: What is th